--- Day 7: Bridge Repair ---

The Historians take you to a familiar rope bridge over a river in the middle of a jungle. The Chief isn't on this side of the bridge, though; maybe he's on the other side?

When you go to cross the bridge, you notice a group of engineers trying to repair it. (Apparently, it breaks pretty frequently.) You won't be able to cross until it's fixed.

You ask how long it'll take; the engineers tell you that it only needs final calibrations, but some young elephants were playing nearby and stole all the operators from their calibration equations! They could finish the calibrations if only someone could determine which test values could possibly be produced by placing any combination of operators into their calibration equations (your puzzle input).

For example:

```
190: 10 19
3267: 81 40 27
83: 17 5
156: 15 6
7290: 6 8 6 15
161011: 16 10 13
192: 17 8 14
21037: 9 7 18 13
292: 11 6 16 20
```

Each line represents a single equation. The test value appears before the colon on each line; it is your job to determine whether the remaining numbers can be combined with operators to produce the test value.

Operators are always evaluated left-to-right, not according to precedence rules. Furthermore, numbers in the equations cannot be rearranged. Glancing into the jungle, you can see elephants holding two different types of operators: add (+) and multiply (*).

Only three of the above equations can be made true by inserting operators:

* 190: 10 19 has only one position that accepts an operator: between 10 and 19. Choosing + would give 29, but choosing * would give the test value (10 * 19 = 190).
* 3267: 81 40 27 has two positions for operators. Of the four possible configurations of the operators, two cause the right side to match the test value: 81 + 40 * 27 and 81 * 40 + 27 both equal 3267 (when evaluated left-to-right)!
* 292: 11 6 16 20 can be solved in exactly one way: 11 + 6 * 16 + 20.

The engineers just need the total calibration result, which is the sum of the test values from just the equations that could possibly be true. In the above example, the sum of the test values for the three equations listed above is 3749.

Determine which equations could possibly be true. What is their total calibration result?

In [104]:
# @author: kaenkai

import re
# import typing


TestEqns = '''
190: 10 19
3267: 81 40 27
83: 17 5
156: 15 6
7290: 6 8 6 15
161011: 16 10 13
192: 17 8 14
21037: 9 7 18 13
292: 11 6 16 20
'''[1:-1]


def verifyEq(eqn: str) -> bool:
    """ Verifies equation
    Args:
        eqn (str): equation
    Return:
        (bool): is a valid equation or not
    """
    r, *n = eqn
    opCount = len(n)-1
    for i in range(int('1'*opCount, 2)+1):
        # Count operators = count numbers - 1; 0 --> +, 1 --> *
        op = f'{i:0{opCount}b}'
        result = n[0]
        for ii in range(len(op)):
            if op[ii] == '0':
                result += n[ii+1]
            elif op[ii] == '1':
                result *= n[ii+1]
        if result == r:
            return True
    return False


def main(inputFile: str) -> int:
    """Main function

    Args:
        inputFile (str): input file name
    """
    print('Running main function ...')
    
    try:
        with open(inputFile, 'r') as fin:
            eqns = fin.read()[:-1]
    except FileNotFoundError as err:
        print(err)
        print('Using test data.')
        eqns = TestEqns
    eqns = [[int(n) for n in re.findall(r'[0-9]+', e)] for e in eqns.split('\n')]
    calibration = 0
    for eqn in eqns:
        calibration += eqn[0] if verifyEq(eqn) else 0
    print(f'\nResult: {calibration:_d}')


if __name__ == '__main__':
    main('input_day7.txt')
    # 12_839_601_725_877


Running main function ...

Result: 12_839_601_725_877


--- Part Two ---

The engineers seem concerned; the total calibration result you gave them is nowhere close to being within safety tolerances. Just then, you spot your mistake: some well-hidden elephants are holding a third type of operator.

The concatenation operator (||) combines the digits from its left and right inputs into a single number. For example, 12 || 345 would become 12345. All operators are still evaluated left-to-right.

Now, apart from the three equations that could be made true using only addition and multiplication, the above example has three more equations that can be made true by inserting operators:

* 156: 15 6 can be made true through a single concatenation: 15 || 6 = 156.
* 7290: 6 8 6 15 can be made true using 6 * 8 || 6 * 15.
* 192: 17 8 14 can be made true using 17 || 8 + 14.

Adding up all six test values (the three that could be made before using only + and * plus the new three that can now be made by also using ||) produces the new total calibration result of 11387.

Using your new knowledge of elephant hiding spots, determine which equations could possibly be true. What is their total calibration result?



In [65]:
# @author: kaenkai

import re


TestEqns = '''
190: 10 19
3267: 81 40 27
83: 17 5
156: 15 6
7290: 6 8 6 15
161011: 16 10 13
192: 17 8 14
21037: 9 7 18 13
292: 11 6 16 20
'''[1:-1]

# TestEqns = '7290: 6 8 6 15'


def ternary(n: int) -> str:
    """Coverts a number to ternary system
    Args:
        n (int): an integer number
    Returns:
        (str): num in ternary system
    """
    if n == 0:
        return '0'
    nums: list[str] = []
    while n:
        n, r = divmod(n, 3)
        nums.append(str(r))
    return ''.join(reversed(nums))


def verifyEq(eqn: list[list[int]]) -> bool:
    """ Verifies equation
    Args:
        eqn (str): equation
    Return:
        (bool): is a valid equation or not
    """
    r, *n = eqn
    for i in range(int('2'*(len(n)-1), 3)+1):
        # Count operators = count numbers - 1;
        # 0 --> +, 1 --> *, 2 --> ||
        opMath = f'{ternary(i):0>{len(n)-1}}'
        n_ = n.copy()
        if len(opMath) == 1 and opMath[0] == '2':
            result = int(f'{n[0]}{n[1]}')
        else:
            result = n[0]
            for ii in range(len(opMath)):
                if opMath[ii] == '0':
                    result += n[ii+1]
                elif opMath[ii] == '1':
                    result *= n[ii+1]  
                elif opMath[ii] == '2':
                    result = int(f'{result}{n[ii+1]}')
        if result == r:
            return True
    return False


def main(inputFile: str) -> int:
    """Main function

    Args:
        inputFile (str): input file name
    """
    print('Running main function ...')
    
    try:
        with open(inputFile, 'r') as fin:
            eqns = fin.read()[:-1]
    except FileNotFoundError as err:
        print(err)
        print('Using test data.')
        eqns = TestEqns
    eqns = [[int(n) for n in re.findall(r'[0-9]+', e)] for e in eqns.split('\n')]
    calibration = 0
    for eqn in eqns:
        calibration += eqn[0] if verifyEq(eqn) else 0
    print(f'\nResult: {calibration:d}')


if __name__ == '__main__':
    main('input_day7.txt')
    # 149956401519484


Running main function ...

Result: 149956401519484
